#### Importar bibliotecas e funcionalidades

In [5]:
import os
import pandas as pd

#### Visualisar colunas das planilhas

In [6]:
current_dir = os.getcwd()
parent_dir = os.path.abspath(os.path.join(current_dir, os.pardir))

path = f"{parent_dir}\\data_cleaned"

cols_by_archive = {}
for archive in os.listdir(path):
    if archive[:5] == "clean":
        df = pd.read_csv(f"{path}\\{archive}")
        cols_by_archive[archive] = df.columns.to_list()
        
print(cols_by_archive)

{'clean_dim_familia_produtos.csv': ['codigo_familia', 'descricaofamilia'], 'clean_dim_produtos.csv': ['codigo_produto', 'descricaoproduto', 'codigo_familia', 'custo_produto_unitario'], 'clean_dim_vendedor.csv': ['codigo_vendedor', 'nome_vendedor'], 'clean_fato_vendas.csv': ['codigo_contrato', 'data_venda', 'codigo_produto', 'filial_venda', 'quantidade', 'valor_monetario_total', 'valor_desconto', 'codigo_vendedor', 'codigo_cliente']}


C:\Users\diogo\AppData\Local\Temp\ipykernel_3144\3546317070.py:9: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f"{path}\\{archive}")


#### Ler os arquivos(planilhas)


In [7]:
data_dir = path
df_fato_vendas = pd.read_csv(f"{data_dir}\\clean_fato_vendas.csv")
df_dim_vendedor = pd.read_csv(f"{data_dir}\\clean_dim_vendedor.csv")
df_dim_produtos = pd.read_csv(f"{data_dir}\\clean_dim_produtos.csv")
df_dim_familia_produtos = pd.read_csv(f"{data_dir}\\clean_dim_familia_produtos.csv")

C:\Users\diogo\AppData\Local\Temp\ipykernel_3144\2487375757.py:2: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df_fato_vendas = pd.read_csv(f"{data_dir}\\clean_fato_vendas.csv")


#### Agrupar e mesclar planilhas por colunas iguais


In [8]:
output_path = f"{parent_dir}\\data_related"

# Primeiro, agrupar e somar os valores monetários e quantidade por produto
df_sums = df_fato_vendas.groupby("codigo_produto", as_index=False)[["valor_monetario_total", "quantidade"]].sum()

# Selecionar apenas as colunas necessárias para os merges
df_dim_produtos_reduced = df_dim_produtos[["codigo_produto", "codigo_familia", "custo_produto_unitario", "descricaoproduto"]]
df_dim_familia_produtos_reduced = df_dim_familia_produtos[["codigo_familia", "descricaofamilia"]]

# Realizar o merge com df_dim_produtos
df_sums = pd.merge(df_sums, df_dim_produtos_reduced, on="codigo_produto", how="inner")

# Realizar o merge com df_dim_familia_produtos
df_sums = pd.merge(df_sums, df_dim_familia_produtos_reduced, on="codigo_familia", how="inner")

# Calcular o lucro por produto
df_sums["lucro_produto"] = df_sums["valor_monetario_total"] - (df_sums["quantidade"] * df_sums["custo_produto_unitario"])

# Salvar o resultado final como CSV
output_dir = os.path.join(output_path, "df_fam_prod_venda.csv")
df_sums.to_csv(output_dir, index=False)

print(f"Arquivo salvo em: {output_dir}")


Arquivo salvo em: g:\Meu Drive\2. Creating\3. Projetos\1. Data Science\Python Datasets\Python Case Equal\data_related\df_fam_prod_venda.csv
